# **GPT4ALL QA from docs:**
**Para analizar grandes cantidades de datos el precio de OpenAI deja de ser trivial, por lo tanto se explora la posibilidad de usar modelos en local que realicen lo mismo pero con el nivel más parecido al SotA actual.
Es por esto que se intenta usar [GPT4ALL](https://github.com/nomic-ai/gpt4all) y su implementación mediante librería [Langchain](https://python.langchain.com/en/latest/index.html#)**

**Sin embargo, los embeddings de OPENAI siguen siendo baratos y efectivos, por lo que su uso sí tiene sentido**

In [6]:
# Imports and setup
import numpy as np
import pandas as pd
import csv
import os
import re
import io
import json
import pypdf
import langchain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
import pyllamacpp
import tabula
import openai

api_key = json.load(open('./data/creds/gpt_id.json'))['api_key']
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

In [7]:
# Funciones esenciales de procesamiento:
def limpieza_texto(texto: str) -> str:
    '''
    Función para limpiar texto de pdfs.
    Cambia saltos de línea, espacios en blanco y caracteres especiales.
    '''
    # Eliminamos espacios en blanco
    #texto = re.sub(' +', ' ', texto)
    # Eliminamos caracteres especiales [REVISAR]
    #texto = re.sub('[^A-Za-z0-9]+', ' ', texto)
    # Eliminamos saltos múltiples de línea
    texto = re.sub(r"\n\s*\n", "\n\n", texto)
    texto = re.sub(r"\n", ";", texto)
    return texto

def tabla_a_texto(tabla):
    '''
    Función para convertir una tabla de pandas en un texto.
    La idea es identificar los nombres de columna e índices correctos y
    a partir de ahí generar un texto que pueda ser procesado por el modelo.
    '''
    tabla = tabla.copy()
    
    # Tamaño mínimo de tabla para que sea válida = 2x2
    if sum(tabla.shape) < 4:
        return ''
    
    # Lista de valores que consideramos NaN:
    nan_equivalents = [np.NaN, np.nan,
                       'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                       'Unnamed:0', 'Unnamed: 0'
                       '', '-', ' ', '  ', '   ']
    
    # Asumimos que el primer elemento es el título salvo si es NaN:
    titulo = tabla.columns[0] if tabla.columns[0] not in nan_equivalents else ''
    
    # Asumimos que la primera columna es el índice y la eliminamos:
    tabla.index = tabla[tabla.columns[0]].values
    tabla.drop(columns=tabla.columns[0], inplace=True)

    # Si las columnas tienen muchos 'Unnamed' suele ser porque hay
    # varias líneas de texto. En ese caso, las juntamos en una sola:
    if sum(['Unnamed' in i for i in tabla.columns]) > 2:
        nueva_columna = [f'{tabla.columns[i]} {tabla.iloc[0,i]}'
                         for i in range(len(tabla.columns))]
        nueva_columna = [i.replace('Unnamed: ','') for i in nueva_columna]
        tabla.columns = nueva_columna

    
    # Eliminamos las filas y columnas que no tienen datos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    
    # Check si las columnas son años:
    col_años = False
    years_txt=[str(i) for i in range(2015,2022)]
    years_int=[i for i in range(2015,2022)]
    years = set(years_txt+years_int)
    cruce = set(tabla.columns).intersection(set(years))
    if len(cruce) > 0: col_años=True
    
    # Si no son años las columnas, buscamos filas que sean años:
    contexto = None
    if not col_años:
        for i in tabla.iterrows():
            #print(i[1].values)
            try:
                cruce = set(i[1].values).intersection(set(years))
            except:
                cruce=[]
            if len(cruce)>0: # Si encontramos una fila con años:
                # Asignamos los años a las columnas:
                tabla.columns = i[1].values
                try: 
                    contexto = i[1].name
                except:
                    contexto = None
                # Drop de la fila:
                tabla.drop(i[0], inplace=True)
                break
    # Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    # Pasamos a texto:
    texto = ''
    for i in tabla.items():
        txt = [f' {titulo} + {i[0]} + {x[0]} = {x[1]}; '
            for x in list(i[1].items())]
        add= ''.join(txt)
        if contexto:
            txt = [f' {titulo} + {contexto} + {i[0]} + {x[0]} = {x[1]}; '
                for x in list(i[1].items())]
            add = ''.join(txt)
        add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
        texto += f';  Tabla={titulo}: {add}'
    return texto

def extract_text_from_pdf(pdf_path: str) -> list:
    '''
    Función para extraer texto de un pdf y limpiarlo.
    Devuelve una lista de str, cada una es una página del pdf.
    '''
    # Abrimos el pdf
    with open(pdf_path, 'rb') as f:
        pdf = pypdf.PdfReader(f)
        # Obtenemos el número de páginas
        num_pags = len(pdf.pages)
        count = 0
        text = []
        # Iteramos sobre las páginas
        for pag in pdf.pages:
            count +=1
            texto_pagina = pag.extract_text()
            tablas = tabula.read_pdf(pdf_path, pages=count)
            for tabla in tablas:
                texto_pagina += f'; {tabla_a_texto(tabla=tabla)}; '
            texto_pagina = limpieza_texto(texto_pagina)
            text.append(texto_pagina)
    return text

In [8]:
# Procesamos el PDF:
resultado = extract_text_from_pdf('./data/rovi-sostenibilidad-2020.pdf')

In [4]:
# Guardamos el resultado en un csv con 1 fila por página
with open('lista_test_2.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerows([[str(i)] for i in resultado])

In [9]:
# Cargamos el csv como una lista (para comprobar que se ha guardado bien):
csv_loaded = list(pd.read_csv('lista_test_2.csv', header=None)[0])


In [3]:
# SPLITTER ESTANDAR:
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function = len)

x = splitter.create_documents(resultado, metadatas=[{'source': f'lista_test_2.csv pag{i}'} for i in list(range(len(resultado)))])

loader = CSVLoader(file_path='lista_test_2.csv')

documentos = loader.load()

In [10]:
# SPLITTER POR TOKENS:
import transformers
tokenizador = transformers.GPT2TokenizerFast.from_pretrained("gpt2")
gpt_splitter = langchain.text_splitter.CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizador, chunk_size=100, chunk_overlap=10)

documentos = gpt_splitter.create_documents(
    csv_loaded,
    metadatas=[{'source': f'lista_test_2.csv pag.{i}'}
               for i in list(range(len(csv_loaded)))])

Token indices sequence length is longer than the specified maximum sequence length for this model (1027 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
# GPT4All CHAIN for QA:
gpt4all_path = './data/models/gpt4all-7B/gpt4all-lora-quantized_new.bin'
llm = langchain.llms.gpt4all.GPT4All(model=gpt4all_path,
                                     n_threads=12)
opai_llm = langchain.llms.openai.OpenAI()
embedding = langchain.embeddings.openai.OpenAIEmbeddings()

In [12]:
vector_store = langchain.vectorstores.Chroma.from_documents(documentos, embedding)

[2023-04-06 19:38:17,563] {posthog.py:15} INFO - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
[2023-04-06 19:38:17,564] {__init__.py:79} INFO - Running Chroma using direct local API.
[2023-04-06 19:38:17,580] {__init__.py:48} WARNING - Using embedded DuckDB without persistence: data will be transient
[2023-04-06 19:38:17,594] {ctypes.py:22} INFO - Successfully imported ClickHouse Connect C data optimizations
[2023-04-06 19:38:17,595] {ctypes.py:31} INFO - Successfully import ClickHouse Connect C/Numpy optimizations
[2023-04-06 19:38:17,603] {json_impl.py:45} INFO - Using python library for writing JSON byte strings


In [13]:
recuperador = vector_store.as_retriever()

In [19]:
# OPENAI:
cadena_OAI = langchain.chains.question_answering.load_qa_chain(llm=opai_llm, reduce_k_below_max_tokens=True)

# GPT4ALL:
cadena_GPT4all = langchain.chains.question_answering.load_qa_chain(llm=llm)

# GPT4ALL Alternativa:
esquema_pregunta = '''Question: {question}
Answer: '''
prompt = langchain.PromptTemplate(
    template = esquema_pregunta,
    input_variables=['question'],
)
cadena_alt = langchain.LLMChain(
    prompt=prompt,
    llm=llm)

ValidationError: 1 validation error for StuffDocumentsChain
reduce_k_below_max_tokens
  extra fields not permitted (type=value_error.extra)

In [15]:
pregunta = 'How much CO2 was emitted in total according to this document?'
similares = vector_store.similarity_search(pregunta, include_metadata=True, top_k=4)
print(similares)

[Document(page_content='are endangered. From a social point of view, the project will contribute to the ;sustainable development of rural producers and indigenous communities that ;live in nearby areas. The project has been certified by FSC (Forest Stewardship ;Council), CCB Gold Level (Climate, Community and Biodiversity) and VCS (Verified ;Carbon Standard).;ATMOSPHERIC EMISSIONS ;2020 2019 Variación Total ;Granada Madrid  ;& SSRR Alcalá de ;Henares Distr.  Granada Madrid  ;& SSRR Alcalá de ;Henares Distr.  Gr Mad  ;y SSRR AH Distr. ;Tonnes of Scope 1 ;CO2 emitted 805 1,494 2,663 836 468 1,399 2,880 1,262 72% 7% -8% -34%;Tonnes of Scope ;2 CO2 emitted 0 0 0 102 1,101 2,245 2,565 179 -100% -100% -100% -43%;Tonnes of Scope ;2 CO2 avoided (*) 1,193 2,198 2,999 96 0 0 0 0 - - - -;Tonnes CO2/ / ;million units. 0.004 22.58 104.85 42.30 0.003 27.54 137.86 71.42 38% -18% -24% -41%305-1;91; ;  Tabla=:  + 1 Granada + nan = Granada;+ 1 Granada + CO2 emitted = 805;+ 1 Granada + 2 CO2 emitted = 0;

In [18]:
# OPENAI:
cadena_OAI.run(
    input_documents = similares,
    question = pregunta)  

[2023-04-06 19:38:39,377] {util.py:67} INFO - error_code=None error_message="This model's maximum context length is 4097 tokens, however you requested 5022 tokens (4766 in your prompt; 256 for the completion). Please reduce your prompt; or completion length." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5022 tokens (4766 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [16]:
# GPT4ALL:
cadena_GPT4all.run(
    input_documents = similares,
    question = pregunta)  

''

In [17]:
# GPT4ALL con Cadena Alternativa:
cadena_alt.run(
    input_documents = similares,
    question = pregunta)

' Question: How much CO2 was emitted in total according to this document?\nAnswer: 3.8 trillion metric tons of carbon dioxide were released into the atmosphere from human activities since preindustrial times, contributing greatly to climate change and global warming'

In [67]:
similares

[Document(page_content='REPORT;20;20: 98www.rovi.es', metadata={'source': 'lista_test_2.csv', 'row': 96}),
 Document(page_content='REPORT;20;20: SHAREHOLDER COMPOSITION;63.11%;Norbel Inversores, S.L.;5.57%;Indumenta Pueri, S.L.;3.043%;T.Rowe Price International Funds, Inc;3.005%;Wellington Management Group, LLP;25.275%;Other;7', metadata={'source': 'lista_test_2.csv', 'row': 5}),
 Document(page_content='REPORT;20;20: –Profarma;Each year, in the Plan Profarma, the Ministry of Industry, Tourism and Trade and the Ministry ;of Health, Social Services and Equality classify the pharmaceutical Companies in accordance ;with their contribution to the Spanish industrial fabric, taking their investment in technology, ;new manufacturing plants, research efforts, etc. as a reference. In February 2020, the results ;of Plan Profarma 2019 were issued and ROVI obtained the classification of Excellent for the ;fourteenth consecutive year. ;KEY FIGURES;(million euros) 2020 2019 2018 2017;Total revenue 42

In [ ]:
pregunta = '¿De qué empresa es este informe?'
similares = recuperador.get_relevant_documents(pregunta)
cadena.run(input_documents = similares, question = pregunta)  

''

In [5]:
from langchain.embeddings import LlamaCppEmbeddings
LlamaCppEmbeddings(model_path='./data/models/gpt4all-7B/gpt4all-lora-quantized_new.bin')

NameError: Could not load Llama model from path: ./data/models/gpt4all-7B/gpt4all-lora-quantized_new.bin